In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!unzip -q "/content/gdrive/MyDrive/Colab/Assignment/Data/archive.zip" -d /content/Data

In [11]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip -q install gputil

import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]

print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

GPU RAM Free: 15857MB | Used: 423MB | Util   3% | Total 16280MB


In [4]:
from tensorflow.keras.layers.experimental import preprocessing as pl
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB1
import tensorflow.keras.layers as l
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from PIL import Image
import pandas as pd
import numpy as np
import random
import json
import os

In [89]:
# Hyperparameters and directories

GDRIVE_PROJ_DIR = "/content/gdrive/MyDrive/Colab/Assignment"
TRAIN_DIR = "/content/Data/tiny-imagenet-200/train"
TEST_DIR = "/content/Data/tiny-imagenet-200/val"

model_json_location = os.path.join(GDRIVE_PROJ_DIR, "effnet_architecture_64.json")

"""
Setting 1 = last_epoch=0,epochs=30,img_size=16,batch_size=2024,learning_rate=1e-3,dropout_scaler=2.6 
Setting 2 = last_epoch=30,epochs=120,img_size=32,batch_size=756,learning_rate=1e-2, dropout_scaler=3
Setting 3 = last_epoch=150,epochs=5,img_size=64,batch_size=128,learning_rate=1e-2, dropout_scaler=2.25 :- acc tr=41.76 val=36.82
Setting 4 = last_epoch=155,epochs=45,img_size=64,batch_size=256,learning_rate=1e-2, dropout_scaler=2.25 :- acc tr=54.81 val=44.48 #log not available
Setting 5 = last_epoch=200,epochs=20,img_size=64,batch_size=256,learning_rate=1e-2, dropout_scaler=2.6 :- acc tr=59.91 val=45.79
Setting 5 = last_epoch=220,epochs=20,img_size=64,batch_size=256,learning_rate=1e-2, dropout_scaler=2.6 :- acc tr=NA val=NA
"""

last_epoch     =  200
epochs         =  20
img_size       =  64
batch_size     =  256
learning_rate  =  1e-2

l2_reg_rate    =  None

# dropout_scaler - to multiply original dropout rates by this factor
# dropout_abs - to set absoulte value to dropout layers
# Don't keep both always put one as None
# if both are not None dropout_abs will override
dropout_scaler =  2       # Max = 5 (very hard dropout)
dropout_abs    =  None

In [90]:
train_csv = pd.read_csv(os.path.join(GDRIVE_PROJ_DIR, 'train.csv'))
train_csv.head()

,image_path,class_code,class_name
0,n03126707/images/n03126707_344.JPEG,n03126707,crane
1,n02437312/images/n02437312_328.JPEG,n02437312,"Arabian camel, dromedary, Camelus dromedarius"
2,n02113799/images/n02113799_124.JPEG,n02113799,standard poodle
3,n03980874/images/n03980874_261.JPEG,n03980874,poncho
4,n03649909/images/n03649909_208.JPEG,n03649909,"lawn mower, mower"


In [91]:
# Creating input Pipeline
datagen = ImageDataGenerator(validation_split=0.2)
train_gen = datagen.flow_from_dataframe(
    train_csv, directory=TRAIN_DIR, x_col='image_path', y_col='class_name',
    weight_col=None, target_size=(img_size, img_size), class_mode='categorical', 
    batch_size=batch_size, shuffle=True,subset="training")
val_gen = datagen.flow_from_dataframe(
    train_csv, directory=TRAIN_DIR, x_col='image_path', y_col='class_name',
    weight_col=None, target_size=(img_size, img_size), class_mode='categorical', 
    batch_size=batch_size, shuffle=False,subset="validation")

Found 80000 validated image filenames belonging to 200 classes.
Found 20000 validated image filenames belonging to 200 classes.


In [92]:
with open(model_json_location, 'r') as file:
    model_json=json.load(file)

backbone = tf.keras.models.model_from_json(model_json)

In [93]:
# Update regulariztion parameters of base netowrk
reg_updated = False
drp_rate_updated = False
for layer in backbone.layers:
    layer_name = layer.name
    if "conv" in layer_name and "pad" not in layer_name:
        if l2_reg_rate!=None:
            layer.activity_regularizer = tf.keras.regularizers.L2(l2=l2_reg_rate)
            reg_updated=True
    if "drop" in layer_name:
        if dropout_scaler != None:
            layer.rate *= dropout_scaler
            drp_rate_updated = True
        if dropout_abs != None:
            layer.rate = dropout_abs
            drp_rate_updated = True

if drp_rate_updated:
    print("dropout rate updated")
if reg_updated:
    print("regularization updated")

dropout rate updated


In [94]:
# Build Modle
Augument_pipeline = tf.keras.Sequential([
                                        pl.RandomRotation(factor=0.15),
                                        pl.RandomTranslation(height_factor=0.1, width_factor=0.1),
                                        pl.RandomFlip(),
                                        pl.RandomContrast(factor=0.1),
                                        ], name="Image_Augumentation")

model = tf.keras.Sequential([
                             l.Input(shape = (img_size,img_size,3)),
                             Augument_pipeline,
                             backbone,
                             l.GlobalAveragePooling2D(name="Global_Average_pooling"),
                             l.Dense(200, activation="softmax", name="Classifier")
                            ], name="Effnet_Classifier") 
model.summary()

Model: "Effnet_Classifier"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Image_Augumentation (Sequent (None, 64, 64, 3)         0         
_________________________________________________________________
efficientnetb1 (Functional)  (None, 4, 4, 1280)        6575239   
_________________________________________________________________
Global_Average_pooling (Glob (None, 1280)              0         
_________________________________________________________________
Classifier (Dense)           (None, 200)               256200    
Total params: 6,831,439
Trainable params: 6,769,384
Non-trainable params: 62,055
_________________________________________________________________


In [95]:
loss = tf.keras.losses.CategoricalCrossentropy()
adam_opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
opt = tfa.optimizers.MovingAverage(adam_opt)
metrics = [tf.keras.metrics.CategoricalAccuracy()]

temp_dir = os.path.join(GDRIVE_PROJ_DIR, 'temp', "Efficient_best.h5")

callbacks = [
                tfa.callbacks.AverageModelCheckpoint(filepath=temp_dir,update_weights=True, save_best_only=True),
                tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.3, patience=5, verbose=1)
            ]

model.compile(loss=loss, optimizer=opt, metrics=metrics)

In [96]:
# Load Last Checkpoint /content/gdrive/MyDrive/Colab/Assignment/models/Effnetnet_64_201_220.h5
model.load_weights(os.path.join(GDRIVE_PROJ_DIR, "models", "Effnetnet_64_201_220.h5"))

In [ ]:
model_logs = model.fit(
    initial_epoch=last_epoch,
    x=train_gen, validation_data=val_gen,
    steps_per_epoch=train_gen.samples//batch_size,
    validation_steps=val_gen.samples//batch_size,
    epochs=last_epoch+epochs, verbose=1, shuffle=True,
    callbacks=callbacks
)
model.save(os.path.join(GDRIVE_PROJ_DIR, 'temp', "Efficient_last.h5"))

Epoch 201/220
312/312 [==============================] - 263s 754ms/step - loss: 1.8909 - categorical_accuracy: 0.5139 - val_loss: 2.8662 - val_categorical_accuracy: 0.3766
Epoch 202/220
312/312 [==============================] - 231s 741ms/step - loss: 1.8606 - categorical_accuracy: 0.5205 - val_loss: 3.0132 - val_categorical_accuracy: 0.3527
Epoch 203/220
312/312 [==============================] - 234s 749ms/step - loss: 1.8457 - categorical_accuracy: 0.5252 - val_loss: 2.8778 - val_categorical_accuracy: 0.3764
Epoch 204/220
225/312 [====================>.........] - ETA: 1:02 - loss: 1.8097 - categorical_accuracy: 0.5333

In [97]:
#model.save("Efficient_last.h5")
model.evaluate(train_gen,steps=train_gen.samples//batch_size)
model.evaluate(val_gen,steps=val_gen.samples//batch_size)

78/78 [==============================] - 10s 122ms/step - loss: 2.2754 - categorical_accuracy: 0.4579


[2.27536940574646, 0.4578826129436493]

In [ ]:
model.load_weights(temp_dir)
model.evaluate(train_gen,steps=train_gen.samples//batch_size)
model.evaluate(val_gen,steps=val_gen.samples//batch_size)

In [ ]:
def plot_metrics(ax, history,title, metrics):
    ax.plot(history.history[metrics], label=metrics)
    ax.plot(history.history["val_"+metrics], label="val_"+metrics)
    ax.set(xlabel="epochs", ylabel=title)
    ax.set_title(title)
    ax.legend()

metrics_plots = len(model_logs.history.keys())//2
fig, axes = plt.subplots(1,metrics_plots, figsize=(20,5))
plt.subplots_adjust(hspace = 0.3)

metrics_plot = list(model_logs.history.keys())[:metrics_plots]

for i in range(len(metrics_plot)):
    plot_metrics(axes[i], model_logs, metrics_plot[i], metrics_plot[i])

In [ ]:
# Confusion Matrix 
predicted_labels = model.predict(val_gen,steps=1+val_gen.samples//batch_size,verbose=1)
predicted_labels = np.argmax(predicted_labels, axis=-1)
true_labels = np.array(val_gen.classes)
confus_mat = tf.math.confusion_matrix(true_labels, predicted_labels).numpy()
fig, ax = plt.subplots(figsize=(25,25))
sns.heatmap(confus_mat, ax=ax)

In [ ]:
# calculate per class accuracy
k = 10

# Get indices to label map
label_map = {}
for key,value in train_gen.class_indices.items():
    label_map[value]=key

# Get top k classes
total_per_class=np.sum(confus_mat, axis=-1)
correct_per_class = [confus_mat[i,i] for i in range(200)]
accuracy_per_class = (correct_per_class/total_per_class)*100
sorted_per_acc = accuracy_per_class.argsort().tolist()[::-1]

top_k = [["label", "index", "accuracy"]]
for i in sorted_per_acc[:k]:
    top_k.append([label_map[i], i, round(accuracy_per_class[i],2)])

for row in top_k:
    print("{: <100} {: <10} {: <30}".format(*row))

In [ ]:
img_row = 5
img_col = 7
images_to_visualize = img_row*img_col

# Visualize some of the test examples

images = os.listdir(os.path.join(TEST_DIR,"images"))
images = [random.choice(images) for _ in range(images_to_visualize+10)]
img_iter = iter(images)

fig, ax = plt.subplots(img_row,img_col, figsize=(21,15))
plt.subplots_adjust(hspace=0.5)

for i in range(img_row):
    for j in range(img_col):
        image = next(img_iter)
        pil_img = Image.open(os.path.join(TEST_DIR,"images",image), 'r')
        try:
            img_arr = np.expand_dims(np.asarray(pil_img), axis=0)
            class_idx = np.argmax(model.predict_step(img_arr))
        except:
            print(image, img_arr.shape)
        class_pred = label_map[class_idx]
        ax[i][j].imshow(np.asarray(pil_img))
        ax[i][j].title.set_text(class_pred.split(', ')[0])

In [ ]:
#  confirm before logging to avoid overwriting
log_dir = os.path.join(GDRIVE_PROJ_DIR,"Logs", f"EffNET_train_log_{img_size}_{last_epoch+1}_{last_epoch+epochs}.csv")
model_dir = os.path.join(GDRIVE_PROJ_DIR, "models", f"Effnetnet_{img_size}_{last_epoch+1}_{last_epoch+epochs}.h5")

print(log_dir)
print(model_dir)

In [ ]:
import shutil

# Log Results to Drive 
model_logs.history["epoch"] = [x for x in range(last_epoch+1,last_epoch+epochs+1)]
log_df = pd.DataFrame(model_logs.history)
log_df.to_csv(log_dir, index=False)

shutil.copy(os.path.join(GDRIVE_PROJ_DIR, 'temp', "Efficient_last.h5"), model_dir)
log_df.head()